In [2]:
!pip3 install diffusers

In [ ]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()


In [9]:
import torch
from torchvision import transforms 
from torch.utils.data import DataLoader
from torch.nn.functional import mse_loss
from torch.optim import AdamW
from diffusers import UNet2DModel, DDPMScheduler , DiffusionPipeline
from diffusers.optimization import get_cosine_schedule_with_warmup
from datasets import load_dataset 
from accelerate import Accelerator
from PIL import Image
from tqdm import tqdm 
import matplotlib.pyplot as plt
import numpy as np 
import random 
import timeit 



In [14]:
RANDOM_SEED= 42
IMG_SIZE = 64
BATCH_SIZE = 128
LEARNING_RATE = 1e-4
NUM_EPOCHS = 20
NUM_GENERATE_IMAGES=9
NUM_TIMESTEPS=1000
MIXED_PRECISION="fp16"  
GRADIENT_ACCUMULATION_STEPS=1

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark=False

device = "cuda" if torch.cuda.is_available() else "cpu"


In [11]:
dataset= load_dataset("huggan/smithsonian_butterflies_subset", split="train")

In [12]:
fig , axs =plt.subplots(1,4, figsize=(16,4))
for i, image in enumerate(dataset[:4]["image"]):
    axs[i].imshow(image)
    axs[i].set_axis_off()
fig.show()



In [15]:
preprocess = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

In [16]:
def transform(examples):
    image = [preprocess(image.convert("RGB")) for image in examples["images"]]
    return {"images": images}

dataset.set_transform(transform)

In [ ]:
train_dataloader = DataLoader(dataset)